## Make Tim2DDriver work
- For `observe!` from `Tim2DDriver` to work, had to make the following changes
    - Had to modify `AutomotiveDrivingModels.jl/src/2d/vehicles/scene_records.jl` to extend
the default length of the capacity in the record container from 100 to 300
    - Did the same capacity extension in `Records.jl/src/frames.jl`, `QueueRecords.jl` and 
`ListRecords.jl`
- Note that querying Tim2DDriver yields a `latLonAccel` return type. Had to create an array
to enable these values to work with `step`

## Changes to environment file
- `LatLonAccel` instead of `AccelTurnRate` because ego vehicles are
driven using Tim2DDriver that outputs lat lon accel
- Changed the render method to not save images in `/tmp` but instead
return the render frame to be able to make videos using Reel

In [1]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # Make video in notebook
using Reel # Save video as gif
using CSV # For writing to csv
using DataFrames # For writing to csv
using PyPlot # For in notebook plotting

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5988728129666970501\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-10640478552864238211\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/edf674b4a352d3049d1cf8e1b8f9eda882001cd6-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-5988728129666970501\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

In [ ]:
using AutoEnvs

n_veh = 1 # Number of ego vehicles
# filepath = joinpath(dirname(pathof(NGSIM)), "..", 
#     "data", "trajdata_i80_trajectories-0400-0415.txt")
filepath = joinpath(dirname(pathof(NGSIM)), "..", 
    "data", "trajdata_i101_trajectories-0750am-0805am.txt")
params = Dict(
        "trajectory_filepaths"=>[filepath],
        "H"=>200,
        "primesteps"=>50,
        "n_veh"=>n_veh,
        "remove_ngsim_veh"=>false
)
# env = MultiagentNGSIMEnvVideoMaker(params)
env = MultiagentNGSIMEnv(params);
timestep = 0.1;

In [ ]:
# Sanity check: See if what we have loaded makes sense by rendering a scene
# Test like this is what helped catch the NGSIM occursin order reversing issue
randi = 5
reset(env,random_seed=randi) # Returns an array of features 1x66 
render(env.scene,env.roadway)

In [ ]:
env.scene.entities[1].state.posF.t

In [ ]:
env.scene.n

In [ ]:
env.t

In [ ]:
model = Tim2DDriver(timestep,mlane = MOBIL(timestep));

In [ ]:
# Borrowed from `sisl/gail-driver/validation/validation.jl`
# Reduced T to 0.1 to see more aggressive driving
# Now wondering how to make the blue vehicle change lanes
mlon = IntelligentDriverModel(v_des = 20.0, σ=2.5,k_spd=1.0,T=0.5,s_min=2.0,a_max=3.0,d_cmf=2.5)
mlat = ProportionalLaneTracker(σ=0.1, kp=3.0, kd=2.0)
mlane = MOBIL(timestep,politeness=0.0,advantage_threshold=0.0,safe_decel=3.0)
#mlane = TimLaneChanger(timestep)
model = Tim2DDriver(timestep, mlon=mlon, mlat=mlat, mlane=mlane);

In [ ]:
# TODO: How to define a function with only kwargs and no args?
function run_one_sim(numsteps)
    @show numsteps
    traj = zeros(numsteps,n_veh,66) # 66 observations
    
    data_rmse_t = zeros(numsteps,2)
    data_rmse_vel = zeros(numsteps,2)
    data_rmse_pos = zeros(numsteps,2)
    
    hard_brake_sum = 0
    
    for ii in 1:numsteps
    
        a = zeros(env.n_veh,2)
        
        for (jj,veh) in enumerate(env.ego_vehs)
            observe!(model,env.scene,env.roadway,veh.id)
            latlonacc = rand(model)
            a[jj,1] = latlonacc.a_lat
            a[jj,2] = latlonacc.a_lon
        end
        
        x, r, terminal, info = step(env, a)
        
#         @show info # info contains the rmse information
        
        # Extract rmse information from simulation
        vec_rmse_t = info["rmse_t"]
        vec_rmse_pos = info["rmse_pos"]
        vec_rmse_vel = info["rmse_vel"]
        
        # Compute means over all the agents in this timestep
        mean_rmse_t = mean(vec_rmse_t)
        mean_rmse_vel = mean(vec_rmse_vel)
        mean_rmse_pos = mean(vec_rmse_pos)
        
        # Store rmse into array where column 1 is time and col 2 is the value
        data_rmse_t[ii,1] = ii-1; data_rmse_t[ii,2] = mean_rmse_t
        data_rmse_vel[ii,1] = ii-1; data_rmse_vel[ii,2] = mean_rmse_vel
        data_rmse_pos[ii,1] = ii-1; data_rmse_pos[ii,2] = mean_rmse_pos
        
        
        
        traj[ii,:,:] = x # x is the features: its num_ego_veh x 66
        
#         @show sum(x[:,19]) # 19th column of x captures the is colliding 1 or 0
        @show size(x)
        @show "yo"
        accel_vec = x[:,9]
        
        hard_brake_sum += sum(accel_vec .< -1.0)
        
        dleft_vec = x[:,21]; @show dleft_vec
        dright_vec = x[:,22]; @show dright_vec
        @show min(dleft_vec,dright_vec)
        
        @show find(min(x[:,21],x[:,22])<=-1.0)
    end
    
    # Plotting the rmse stuff
#     figure(0)
#     plot(data_rmse_pos[:,2])
#     figure(1)
#     plot(data_rmse_t[:,2])
#     figure(2)
#     plot(data_rmse_vel[:,2])
    return traj
end

In [ ]:
randi = 15
reset(env,random_seed=randi)
traj = run_one_sim(1);

In [ ]:
data_rmse_pos

In [ ]:
# If you wanted to write data to CSV files
CSV.write("rmse_t.csv",  DataFrame(data_rmse_t), writeheader=false)
CSV.write("rmse_vel.csv",  DataFrame(data_rmse_vel), writeheader=false)
CSV.write("rmse_pos.csv",  DataFrame(data_rmse_pos), writeheader=false)

In [ ]:
using PyPlot

In [ ]:
plot(data_rmse_pos)

## Video making and saving to file

In [ ]:
# Inspired from the drawsim method in AutoViz/doc/AutoViz.ipynb
function makevid(t,dt=NaN)
    
        a = zeros(env.n_veh,2)

        for (jj,veh) in enumerate(env.ego_vehs)
            observe!(model,env.scene,env.roadway,veh.id)

            latlonacc = rand(model)
            a[jj,1] = latlonacc.a_lat
            a[jj,2] = latlonacc.a_lon
        end
        x, r, terminal, _ = step(env, a)
    render(env)
end

In [ ]:
randi = 5
reset(env,random_seed=randi)
film = roll(makevid, fps=10, duration=5.0)
write("numveh_$(env.n_veh)_seed_$(randi)_timlane.gif",film)

## Video making using recorded cars on ngsim
- Trajdatas is what the car trajectories are stored in
- We want to color the ego vehicle differently to be able to see it

In [2]:
td1 = load_trajdata(1)

scene = Scene(500)
temp_scene = get!(scene,td1,1000)
#render(temp_scene,ROADWAY_101)

Scene(with 125 cars)

In [31]:
# argument 1 loads i101 7:50 to 8:05.
# load_trajdata function defined in NGSIM.jl/src/trajdata.jl
td1 = load_trajdata(1); 

scene = Scene(500)
egoid = [25,5,8,9,10,12,21] # These vehicle ids will be colored blue if the egoid test is performed

# List of the vehicle ids in the scene
# veh.id = 2 5 8 9 10 12 13 14 18 20 21 22 23 25 26 27 31 32 34 35 39 48

# Drive here in the notebook. Replay the trajectory as recorded in the ngsim data
@manipulate for i in 400:500
    temp_scene = get!(scene,td1,i)
    
    carcolors = Dict{Int,Colorant}()
    for veh in temp_scene
        #@show veh.id
        #carcolors[veh.id] = in(veh.id, egoid) ? colorant"blue" : colorant"green"
        
            # Color cars green when they are in segment 3
        if veh.state.posF.roadind.tag.segment == 3
            carcolors[veh.id] = colorant"green"
        end
        
    end
    overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, 
                pos=VecSE2(veh.state.posG.x+0.5,veh.state.posG.y+0.5)) for veh in scene]
    render(temp_scene, ROADWAY_101, 
        #cam=CarFollowCamera{Int}(2,5.0),
        #cam=StaticCamera(VecE2(1966400, 570900), 5.0),
         cam=FitToContentCamera(0.),
        overlays,
        car_colors=carcolors)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope(Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["i"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")))], Dict{Symbol,Any}(:className=>"interact-flex-row-left")), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>500,:min=>400,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()))], Dict{Symbol,Any}(:className=>"interact-flex-row-center")), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")))], Dict{Symbol,Any}(:className=>"interact-flex-row-right"))], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Int64} with 2 listeners. Value:
450, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/home/raunak/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/raunak/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/raunak/.julia/packages/InteractBase/l8cEC/src/../assets/all.js"), Asset("css", nothing, "/home/raunak/.julia/packages/InteractBase/l8cEC/src/../assets/style.css"), Asset("css", nothing, "/home/raunak/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"400\",\"401\",\"402\",\"403\",\"404\",\"405\",\"406\",\"407\",\"408\",\"409\",\"410\",\"411\",\"412\",\"413\",\"414\",\"415\",\"416\",\"417\",\"418\",\"419\",\"420\",\"421\",\"422\",\"423\",\"424\",\"425\",\"426\",\"427\",\"428\",\"429\",\"430\",\"431\",\"432\",\"433\",\"434\",\"435\",\"436\",\"437\",\"438\",\"439\",\"440\",\"441\",\"442\",\"443\",\"444\",\"445\",\"446\",\"447\",\"448\",\"449\",\"450\",\"451\",\"452\",\"453\",\"454\",\"455\",\"456\",\"457\",\"458\",\"459\",\"460\",\"461\",\"462\",\"463\",\"

In [17]:
for i in 150:160
    scene = Scene(500) # Have to create a new scene container every time, wonder how manipulate got by without
    t = get!(scene,td1,i)
    for veh in t
        if veh.state.posF.roadind.tag.segment == 3
            print("$(veh.id) is in segment 3\n")
        else
            print("$(veh.id) not in segment 3\n")
        #display(t.entities[1].state.posF.roadind.tag.segment)
        end
    end
end

2 is in segment 3
4 is in segment 3
5 is in segment 3
6 is in segment 3
8 is in segment 3
9 is in segment 3
10 is in segment 3
12 not in segment 3
13 is in segment 3
14 is in segment 3
18 not in segment 3
20 not in segment 3
21 not in segment 3
22 not in segment 3
23 not in segment 3
25 not in segment 3
26 not in segment 3
27 not in segment 3
31 not in segment 3
32 not in segment 3
34 not in segment 3
35 not in segment 3
37 not in segment 3
39 not in segment 3
40 not in segment 3
43 not in segment 3
44 not in segment 3
47 not in segment 3
48 not in segment 3
49 not in segment 3
51 not in segment 3
53 not in segment 3
54 not in segment 3
55 not in segment 3
56 not in segment 3
60 not in segment 3
62 not in segment 3
71 not in segment 3
74 not in segment 3
2 is in segment 3
4 is in segment 3
5 is in segment 3
6 is in segment 3
8 is in segment 3
9 is in segment 3
10 is in segment 3
12 not in segment 3
13 is in segment 3
14 is in segment 3
18 not in segment 3
20 not in segment 3
21 not in 

In [4]:
display(t1.entities[1].state.posF.roadind)
display(t2.entities[1].state.posF)

UndefVarError: UndefVarError: t1 not defined

In [ ]:
for i in 1:3
    display(t.entities[i].state.posG)
    display(t.entities[i].state.posF)
end